In [1]:
from data import *
from Cluster import *
from redcap import *
import time as t
import pygeoda as pg
import pickle as pkl

## Run only once to save data from the whole country

In [2]:
print('start data preparation')
t1 = t.time()
USA_vote_2004 = percentage_bush_2004()
t2 = t.time()
print('end data preparation '+ str(t2-t1))
print('number of nodes: '+ str(len(USA_vote_2004)))
USA_vote_2004.to_pickle('../data/USA_vote_2004.pkl')

start data preparation


C:\Users\ngolenvaux\Documents\Doctorat\Regionalization\Regionalization_models\src\data.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2004['bush_votes_perc'] = df_2004['candidatevotes'] / df_2004['totalvotes']


end data preparation 42.52929496765137
number of nodes: 3083


In [3]:
graph = Graph(USA_vote_2004.set_index('county_fips',drop=True),['bush_votes_perc'],talk=True)

start first_order
end first_order 1.4849398136138916
start full_order
end full_order 2108.0296964645386
start sort full order
stop sort full order 0.0


In [4]:
FullO_file = open('../data/USA_vote_2004_FullO.pkl','wb')
pkl.dump(graph.FullO_E,FullO_file)
graph.dist.to_csv('../data/USA_vote_2004_dist.csv')

In [5]:
MST = graph.find_MST_full_order_CL()#best_k_regions(k,min_regions,z_min,z_max,n_cut,sc)

start MST
stop MST 132.42447018623352
start Tree arch
stop Tree arch 0.4080657958984375


In [6]:
neighbors_file = open('../data/USA_vote_2004_neighbors.pkl','wb')
pkl.dump(graph.MST.neighbors,neighbors_file)
edges_file = open('../data/USA_vote_2004_edges.pkl','wb')
pkl.dump(graph.MST.edges,edges_file)

## End of the run only once

In [2]:
global_results = pd.read_pickle('../data/comparison_parameters.pkl')
#global_results = pd.DataFrame(columns=['n_nodes','states','k','min_counties','z','sc','heterogeneity','size_regions','regions','n_columns','n_partitions','n_cuts','FiO edges','FuO edges','Sort FuO edges','MST','Tree arch','select regions','column generation'])

In [3]:
redcap_results = pd.read_pickle('../data/redcap_results.pkl')#pd.DataFrame(columns=['n_nodes','states','k','heterogeneity','size_regions','regions','time'])

In [4]:
USA_vote_2004 = pd.read_pickle('../data/USA_vote_2004.pkl')
FullO_file = open("../data/USA_vote_2004_FullO.pkl", "rb")
FullO = pkl.load(FullO_file)
dist = pd.read_csv('../data/USA_vote_2004_dist.csv',index_col=0)
dist.columns = dist.columns.astype(int)
neighbors_file = open('../data/USA_vote_2004_neighbors.pkl','rb')
neighbors = pkl.load(neighbors_file)
edges_file = open('../data/USA_vote_2004_edges.pkl','rb')
edges = pkl.load(edges_file)

In [5]:
def evaluate(graph,k,min_counties,z,sc,output,n_nodes,states):
    t1 = t.time()
    graph.MST.find_regions(min_counties, z, sc, 0, 0)
    t2 = t.time()
    graph.times['column generation'] = t2-t1
    h, x = graph.select_k_regions(k)
    output.loc[len(output)] = [n_nodes,states,k,min_counties,z,sc,h,[len(r) for r in x],x,len(graph.MST.regions[0]),graph.MST.n_partitions,graph.MST.n_cuts,graph.times['FiO edges'],graph.times['FuO edges'],graph.times['Sort FuO edges'],graph.times['MST'],graph.times['Tree arch'],graph.times['select regions'],graph.times['column generation']]
    graph.MST.clear()

def comparate_param(states,n_regions,min_counties,z,sc,output,param):
    graph, n_nodes = None, 0
    if states =='all':
        n_nodes = len(USA_vote_2004)
        graph = Graph(USA_vote_2004.set_index('county_fips',drop=True),['bush_votes_perc'],full_order=(FullO,dist))
        graph.MST = Tree(graph.vertex,neighbors,edges,0,None)
        graph.times['FuO edges'],graph.times['Sort FuO edges'],graph.times['MST'],graph.times['Tree arch'] = 0.0, 0.0, 0.0, 0.0
    else:
        data = percentage_bush_2004(states)
        n_nodes = len(data)
        graph = Graph(data.set_index('county_fips',drop=True),['bush_votes_perc'])
        MST = graph.find_MST_full_order_CL()
    if param == 1:
        for k in n_regions:
            evaluate(graph,k,min_counties,z,sc,output,n_nodes,states)
            print('n_regions = '+str(k)+ ' done')
    elif param == 2:
        for min_c in min_counties:
            evaluate(graph,n_regions,min_c,z,sc,output,n_nodes,states)
            print('min_counties = '+str(min_c)+ ' done')
    elif param == 3:
        for size_z in z:
            evaluate(graph,n_regions,min_counties,size_z,sc,output,n_nodes,states)
            print('z = '+str(size_z)+ ' done')
    elif param == 4:
        for sc_val in sc:
            evaluate(graph,n_regions,min_counties,z,sc_val,output,n_nodes,states)
            print('sc = '+str(sc_val)+ ' done')
        
    

In [6]:
def evaluate_redcap(states,k,output):
    data = percentage_bush_2004(states)
    n_nodes = len(data)
    h, regions, t = redcap(data,k,talk=True)
    output.loc[len(output)] = [n_nodes,states,k,h,[len(regions[x]) for x in regions.keys()],regions,t]


# Test with California State

## k = 5

In [8]:
comparate_param(['CALIFORNIA'],5,[1,3,5,10],1,10,global_results,2)

C:\Users\ngolenvaux\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


min_counties = 1 done
min_counties = 3 done
min_counties = 5 done
min_counties = 10 done


In [10]:
comparate_param(['CALIFORNIA'],5,5,1,[5,10,20,40],global_results,4)

sc = 5 done
sc = 10 done
sc = 20 done
sc = 40 done


In [13]:
comparate_param(['CALIFORNIA'],5,[5,10,20],2,10,global_results,2)

min_counties = 5 done
min_counties = 10 done
min_counties = 20 done


In [16]:
comparate_param(['CALIFORNIA'],5,10,2,[5,10,20,40],global_results,4)

sc = 5 done
sc = 10 done
sc = 20 done
sc = 40 done


In [25]:
comparate_param(['CALIFORNIA'],5,10,3,[10],global_results,4)

sc = 10 done


In [12]:
evaluate_redcap(['CALIFORNIA'],5,redcap_results)

start redcap
end redcap 0.008975744247436523
redcap total heterogeneity :  0.22954071285603542


C:\Users\ngolenvaux\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


## k = 10

In [ ]:
comparate_param(['CALIFORNIA'],10,[3,5,10],1,10,global_results,2)

In [20]:
comparate_param(['CALIFORNIA'],10,5,1,[3,5,10,20],global_results,4)

sc = 3 done
sc = 5 done
sc = 10 done
sc = 20 done


In [22]:
comparate_param(['CALIFORNIA'],10,[3,5,10],2,10,global_results,2)

min_counties = 3 done
min_counties = 5 done
min_counties = 10 done


In [23]:
comparate_param(['CALIFORNIA'],10,5,2,[3,5,10,20],global_results,4)

sc = 3 done
sc = 5 done
sc = 10 done
sc = 20 done


In [13]:
evaluate_redcap(['CALIFORNIA'],10,redcap_results)

start redcap
end redcap 0.007010698318481445
redcap total heterogeneity :  0.12115486320241955


# Test with 5 westerns states

In [28]:
print('start data preparation')
t1 = t.time()
USA_vote_2004 = percentage_bush_2004(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'])
t2 = t.time()
print('end data preparation '+ str(t2-t1))
print('number of nodes: '+ str(len(USA_vote_2004)))

start data preparation
end data preparation 1.1240785121917725
number of nodes: 193


# k = 10

In [36]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],10,[10,20,40],1,10,global_results,2)

min_counties = 10 done
min_counties = 20 done
min_counties = 40 done


In [38]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],10,20,1,[5,10,20],global_results,4)

sc = 5 done
sc = 10 done
sc = 20 done


In [40]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],10,[20,40],2,10,global_results,2)

min_counties = 20 done
min_counties = 40 done


In [43]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],10,40,2,[5,20,40],global_results,4)

sc = 5 done
sc = 20 done
sc = 40 done


In [15]:
evaluate_redcap(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],10,redcap_results)

start redcap
end redcap 0.02501845359802246
redcap total heterogeneity :  0.6183821682399695


## k=5

In [46]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],5,[20,40,80],1,10,global_results,2)

min_counties = 20 done
min_counties = 40 done
min_counties = 80 done


In [48]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],5,40,1,[5,10,20],global_results,4)

sc = 5 done
sc = 10 done
sc = 20 done


In [49]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],5,[40,80],2,10,global_results,2)

min_counties = 40 done
min_counties = 80 done


In [51]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],5,80,2,[5,20,40],global_results,4)

sc = 5 done
sc = 20 done
sc = 40 done


In [16]:
evaluate_redcap(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],5,redcap_results)

start redcap
end redcap 0.02469635009765625
redcap total heterogeneity :  1.3064570319460636


# Test with all US States

## k=10

In [23]:
comparate_param('all',10,[100,300,500],1,50,global_results,2)

C:\Users\ngolenvaux\Documents\Doctorat\Regionalization\Regionalization_models\src\data.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2004['bush_votes_perc'] = df_2004['candidatevotes'] / df_2004['totalvotes']


min_counties = 100 done
min_counties = 300 done
min_counties = 500 done


In [26]:
comparate_param('all',10,200,1,[10,30,100],global_results,4)

C:\Users\ngolenvaux\Documents\Doctorat\Regionalization\Regionalization_models\src\data.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2004['bush_votes_perc'] = df_2004['candidatevotes'] / df_2004['totalvotes']


sc = 10 done
sc = 30 done
sc = 100 done


In [31]:
comparate_param('all',10,[1000],2,10,global_results,2)

min_counties = 1000 done


In [9]:
comparate_param('all',10,400,[(0,4),(2,2)],[2],global_results,4)

sc = 2 done


In [11]:
len(Tree.leaves)

3963

In [15]:
len(Tree.regions[0])

0

In [11]:
Tree.regions = []
Tree.n_cuts = 0
Tree.n_partitions = 0
Tree.he = []

In [24]:
evaluate_redcap('all',10,redcap_results)

start redcap
end redcap 0.76481032371521
redcap total heterogeneity :  21.53039473787628


In [12]:
global_results.loc[50:]

,n_nodes,states,k,min_counties,z,sc,heterogeneity,size_regions,regions,n_columns,n_partitions,n_cuts,FiO edges,FuO edges,Sort FuO edges,MST,Tree arch,select regions,column generation
50,193,"[CALIFORNIA, WASHINGTON, OREGON, IDAHO, NEVADA]",5,80,2,20,1.306457,"[49, 50, 80, 6, 8]","[[16003, 41063, 41001, 53013, 53023, 53021, 53...",41,10,20,0.088343,6.652561,0.022336,0.920932,0.010211,0.130467,1.142926
51,193,"[CALIFORNIA, WASHINGTON, OREGON, IDAHO, NEVADA]",5,80,2,40,1.306457,"[49, 50, 80, 6, 8]","[[16003, 41063, 41001, 53013, 53023, 53021, 53...",41,10,20,0.088343,6.652561,0.022336,0.920932,0.010211,0.123629,1.564219
52,3083,all,10,100,1,50,21.530395,"[398, 212, 71, 445, 1162, 88, 55, 11, 282, 359]","[[48325, 48013, 48255, 48175, 48391, 48409, 48...",263,131,131,1.469064,1636.840292,10.358184,116.588271,0.477663,2.858063,571.967105
53,3083,all,10,300,1,50,21.738180,"[398, 283, 445, 1153, 9, 88, 55, 11, 282, 359]","[[48325, 48013, 48255, 48175, 48391, 48409, 48...",115,57,57,1.469064,1636.840292,10.358184,116.588271,0.477663,2.266970,542.948879
54,3083,all,10,500,1,50,21.738180,"[398, 283, 445, 1153, 9, 88, 55, 11, 282, 359]","[[48325, 48013, 48255, 48175, 48391, 48409, 48...",63,31,31,1.469064,1636.840292,10.358184,116.588271,0.477663,1.759164,498.533915
55,3083,all,10,200,1,10,22.914511,"[474, 55, 194, 592, 293, 359, 398, 147, 136, 435]","[[51113, 51047, 51061, 51177, 51179, 51099, 51...",61,30,30,1.410364,1752.552082,10.153550,116.603161,0.497046,1.081937,291.193209
56,3083,all,10,200,1,30,22.636286,"[398, 283, 445, 681, 375, 55, 194, 11, 282, 359]","[[48325, 48013, 48255, 48175, 48391, 48409, 48...",99,49,49,1.410364,1752.552082,10.153550,116.603161,0.497046,1.380502,291.877214
57,3083,all,10,200,1,100,21.530395,"[398, 212, 71, 445, 1162, 88, 55, 11, 282, 359]","[[48325, 48013, 48255, 48175, 48391, 48409, 48...",177,88,88,1.410364,1752.552082,10.153550,116.603161,0.497046,2.924498,719.866165
58,3083,all,10,1000,2,10,23.555772,"[71, 18, 426, 164, 6, 42, 398, 1112, 194, 652]","[[53077, 53037, 53007, 53047, 53019, 53065, 53...",408660,102209,204420,1.420640,1617.541746,10.259073,138.109255,0.348486,2943.210090,703.047169
59,3083,all,10,1000,2,50,24.669719,"[681, 103, 51, 9, 88, 445, 966, 652, 33, 55]","[[48035, 48425, 48221, 48367, 48497, 48337, 48...",1533,383,766,1.420351,1688.738044,10.355141,115.788785,0.422962,43.519545,8650.308966


In [ ]:
redcap_results

In [14]:
global_results.to_pickle('../data/comparison_parameters.pkl')

In [17]:
redcap_results.to_pickle('../data/redcap_results.pkl')

Other

In [30]:
USA_vote_2004 = pd.read_pickle('../data/USA_vote_2004.pkl')
FullO_file = open("../data/USA_vote_2004_FullO.pkl", "rb")
FullO = pkl.load(FullO_file)
dist = pd.read_csv('../data/USA_vote_2004_dist.csv',index_col=0)
dist.columns = dist.columns.astype(int)
neighbors_file = open('../data/USA_vote_2004_neighbors.pkl','rb')
neighbors = pkl.load(neighbors_file)
edges_file = open('../data/USA_vote_2004_edges.pkl','rb')
edges = pkl.load(edges_file)

In [31]:
graph = Graph(USA_vote_2004.set_index('county_fips',drop=True),['bush_votes_perc'],full_order=(FullO,dist))

start first_order
end first_order 2.020354986190796


In [33]:
graph.MST = Tree(graph.vertex,neighbors,edges,0,None)
graph.MST.h = graph.MST.heterogeneity(graph.vertex)

start Tree arch
stop Tree arch 0.11601805686950684


In [4]:
k=10
min_regions= 300
z_min, z_max = 2, 2
n_cut = 1
sc = 5

In [35]:
print('start finding regions')
t1 = t.time()
graph.MST.find_regions(min_regions, z_min, z_max, n_cut, sc)
t2 = t.time()
print('end finding regions ' + str(t2 - t1))

start finding regions
end finding regions 244.71743035316467


In [36]:
res = graph.select_k_regions(k)

start selecting regions
Academic license - for non-commercial use only - expires 2023-01-28
Using license file C:\Users\ngolenvaux\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3084 rows, 33 columns and 16520 nonzeros
Model fingerprint: 0xc13c0a5d
Variable types: 0 continuous, 33 integer (33 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-03, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 3076 rows and 14 columns
Presolve time: 0.01s
Presolved: 8 rows, 19 columns, 44 nonzeros
Variable types: 0 continuous, 19 integer (19 binary)
Found heuristic solution: objective 25.6809269
Found heuristic solution: objective 25.4645704

Root relaxation: objective 2.533079e+01, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dept

In [41]:
print('start redcap')
t1 = t.time()
votes_2004 = pg.open(USA_vote_2004)
w = pg.queen_weights(votes_2004)
res_redcap = pg.redcap(k,w,USA_vote_2004[['bush_votes_perc']],"fullorder-completelinkage")
t2 = t.time()
print('end redcap '+ str(t2-t1))
clusters_r = {}
for i, c in enumerate(res_redcap['Clusters']):
    if c in clusters_r:
        clusters_r[c] += [USA_vote_2004.loc[i,'FIPS']]
    else:
        clusters_r[c] = [USA_vote_2004.loc[i, 'FIPS']]
h_r = [graph.MST.heterogeneity(clusters_r[i]) for i in clusters_r.keys()]
print('redcap total heterogeneity : ', sum(h_r))

start redcap
end redcap 0.7801389694213867
redcap total heterogeneity :  21.53039473787621


In [52]:
len(neighbors)

3083

In [21]:
dist.columns

Int64Index([27077, 53019, 53065, 53047, 53051, 16021, 30053, 30029, 30035,
            30101,
            ...
            51175, 51083, 51163, 51161, 55083,  4012, 35006, 51199, 51153,
            30031],
           dtype='int64', length=3083)

In [20]:
dist.columns = dist.columns.astype(int)